In [1]:
import pandas as pd
import numpy as np
import pyvista as pv
import os
import calendar
import json

In [2]:
data = pd.read_csv("data/archive/data.csv")

measurements = ["I_131_(Bq/m3)","Cs_134_(Bq/m3)","Cs_137_(Bq/m3)"]

data[measurements] = data[measurements].replace("?", "NaN")
data[measurements] = data[measurements].replace("L", "NaN")
data[measurements] = data[measurements].replace("N", "NaN")

data[measurements] = data[measurements].astype("float64")
data.dropna(inplace=True)

data["loc"] =data["X"].astype("str") + ":" + data["Y"].astype("str")
data["Z"] = 0

display(data)
print(data.dtypes)

,PAYS,Code,Ville,X,Y,Date,End_of_sampling,Duration(h.min),I_131_(Bq/m3),Cs_134_(Bq/m3),Cs_137_(Bq/m3),loc,Z
0,SE,1,RISOE,12.07,55.7,86/04/27,24:00:00,24.0,1.00000,0.00000,0.24000,12.07:55.7,0
1,SE,1,RISOE,12.07,55.7,86/04/28,24:00:00,24.0,0.00460,0.00054,0.00098,12.07:55.7,0
2,SE,1,RISOE,12.07,55.7,86/04/29,12:00,12.0,0.01470,0.00430,0.00740,12.07:55.7,0
3,SE,1,RISOE,12.07,55.7,86/04/29,24:00:00,12.0,0.00061,0.00000,0.00009,12.07:55.7,0
4,SE,1,RISOE,12.07,55.7,86/04/30,24:00:00,24.0,0.00075,0.00010,0.00028,12.07:55.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,HU,24,BUDAPEST,19.10,47.5,86/05/07,08:00,24.0,3.70000,0.58000,1.20000,19.1:47.5,0
2047,HU,24,BUDAPEST,19.10,47.5,86/05/08,08:00,24.0,2.20000,0.33000,0.69000,19.1:47.5,0
2048,HU,24,BUDAPEST,19.10,47.5,86/05/09,08:00,24.0,0.75000,0.14000,0.29000,19.1:47.5,0
2049,HU,24,BUDAPEST,19.10,47.5,86/05/10,08:00,24.0,0.05400,0.00600,0.00900,19.1:47.5,0


PAYS                object
Code                 int64
Ville               object
X                  float64
Y                  float64
Date                object
End_of_sampling     object
Duration(h.min)    float64
I_131_(Bq/m3)      float64
Cs_134_(Bq/m3)     float64
Cs_137_(Bq/m3)     float64
loc                 object
Z                    int64
dtype: object


### Create a pyvista polydata and initialize mesh like that, then save to vtk rather than trying to directly create the vtk file


In [3]:
HEIGHT = 10

for date in data["Date"].unique():
    d = data[data["Date"] == date]

    xyz = d[["X","Y","Z"]].values
    verts = []
    verts.extend(xyz)

    for layer in range(HEIGHT):
        next_layer = xyz.copy()
        next_layer[:, 2] = [layer for _ in range(next_layer.shape[0])]

        verts.extend(next_layer)

    points = pv.PolyData(verts)

    i131 = []
    cs134 = []
    cs137 = []

    for _ in range(HEIGHT + 1):
        i131.extend(d["I_131_(Bq/m3)"].values) 
        cs134.extend(d["Cs_134_(Bq/m3)"].values) 
        cs137.extend(d["Cs_137_(Bq/m3)"].values) 


    assert len(i131) == len(cs134) == len(cs137) == points.n_points

    points.point_data["I131"] = np.array(i131, dtype=np.float64)
    points.point_data["CS134"] = np.array(cs134, dtype=np.float64)
    points.point_data["CS137"] = np.array(cs137, dtype=np.float64)

    if not "data" in os.listdir():
        os.mkdir("data")
        os.mkdir("data/chernobyl-series")
    elif not "chernobyl-series" in os.listdir("data"):
        os.mkdir("data/chernobyl-series")


    file_date = date.replace("\\", "-").replace("/", "-")
    filename = f"chernobyl-{file_date}.vtk"
    points.save(f"data/chernobyl-series/{filename}", binary=False)



In [4]:
def date_to_days(date_str: str) -> float:
    year, month, day = date_str.split("-")

    year = int(year)
    month = int(month)
    day = int(day)

    prev_days = sum([calendar.monthrange(year, m)[1] for m in range(1, month)])

    return float(prev_days + day)


series_files = os.listdir("data/chernobyl-series/")


series_config = {
    "file-series-version" : "1.0",
    "files" : [
        {
            "name" : file,
            "time" : date_to_days(file.split("-", 1)[1].split(".", 1)[0])
        } for file in series_files
    ]
    
}

with open("data/chernobyl-series/chernobyl.series", mode="w+") as series:
    series.write(json.dumps(series_config, indent=4))